<a href="https://colab.research.google.com/github/Jucavilav/CienciaD-taller2/blob/main/Taller2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pylev

In [ ]:
import re
from random import randint
from datetime import datetime
from difflib import SequenceMatcher
import numpy as np
import pandas as pd
import pylev

In [ ]:
pd.set_option("display.max_columns", None)

In [ ]:
pip_df = pd.read_csv("https://raw.githubusercontent.com/Jucavilav/CienciaD-taller2/main/datos/gapminder_final.csv")

In [ ]:
pip_df.shape

(178, 16)

In [ ]:
pip_df.dtypes

country                  object
incomeperperson         float64
alcconsumption          float64
armedforcesrate         float64
breastcancerper100th    float64
co2emissions            float64
femaleemployrate        float64
hivrate                 float64
internetuserate         float64
lifeexpectancy          float64
oilperperson            float64
polityscore             float64
relectricperperson      float64
suicideper100th         float64
employrate              float64
urbanrate               float64
dtype: object

In [ ]:
pip_df.sample(5)

,country,incomeperperson,alcconsumption,armedforcesrate,breastcancerper100th,co2emissions,femaleemployrate,hivrate,internetuserate,lifeexpectancy,oilperperson,polityscore,relectricperperson,suicideper100th,employrate,urbanrate
61,Senegal,561.708585,0.51,0.362261,18.4,1.271087e+08,56.900002,0.90,15.999650,59.318,NaN,7.0,55.794744,6.811439,65.599998,42.38
71,Mongolia,772.933345,3.41,1.211869,6.6,3.009343e+08,50.900002,0.06,12.900005,68.498,NaN,10.0,303.805449,13.089616,52.099998,57.18
53,Spain,15461.758370,11.83,0.979537,50.9,1.082253e+10,41.700001,0.40,65.808554,81.404,1.616859,10.0,1779.503510,5.888479,52.500000,77.12
45,Angola,1381.004268,5.57,1.461329,23.1,2.483580e+08,69.400002,2.00,9.999954,51.093,NaN,-2.0,172.999227,14.554677,75.699997,56.70
104,Cameroon,713.639303,7.90,0.305758,29.7,1.251727e+08,49.000000,5.30,3.999977,51.610,NaN,-4.0,59.551245,7.214221,59.099998,56.76


In [ ]:
pip_df.rename({'country' : 'Pais' , 'incomeperperson' : 'Ingresos por persona' , 'alcconsumption' : 'Consumo de alcohol' , 'armedforcesrate' : 'Tasa de fuerzas armadas' , 'breastcancerper100th' : 'Cáncer de mama100TH' , 'co2emissions' : 'Emisiones de CO2' , 'femaleemployrate' : 'Tasa de empleo femenina' , 'hivrate' : 'Tasa de VIH' , 'internetuserate' : 'Usuario de Internet' , 'lifeexpectancy' : 'Esperanza de vida' , 'oilperperson' : 'Petrolero' , 'polityscore' : 'Puntuación política' , 'relectricperperson' : 'Electricidad' , 'suicideper100th' : 'Suicidio por 100TH' , 'employrate' : 'empleo' , 'urbanrate' : 'tarifa urbana'}, axis=1)

,Pais,Ingresos por persona,Consumo de alcohol,Tasa de fuerzas armadas,Cáncer de mama100TH,Emisiones de CO2,Tasa de empleo femenina,Tasa de VIH,Usuario de Internet,Esperanza de vida,Petrolero,Puntuación política,Electricidad,Suicidio por 100TH,empleo,tarifa urbana
0,Ireland,27595.091350,14.92,0.450024,74.9,1.633779e+09,51.000000,0.2,69.770394,80.557,1.700262,10.0,2051.802338,10.365070,59.900002,61.34
1,Albania,1914.996551,7.29,1.024736,57.4,2.237473e+08,42.099998,NaN,44.989947,76.918,NaN,9.0,636.341383,7.699330,51.400002,46.72
2,Central African Rep.,239.518749,3.17,0.148659,16.5,8.338000e+06,62.900002,4.7,2.300027,48.398,NaN,-1.0,NaN,13.548420,71.300003,38.58
3,Barbados,9243.587053,6.42,0.663956,62.5,3.616067e+07,60.299999,1.4,70.028599,76.835,NaN,NaN,NaN,3.108603,66.900002,39.84
4,Italy,18982.269290,9.72,1.729346,74.4,1.900045e+10,35.799999,0.3,53.740217,81.855,1.207250,10.0,1176.385457,4.930045,46.400002,68.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,Mexico,6105.280743,8.55,0.612340,26.4,1.330450e+10,39.599998,0.3,31.050013,76.954,0.770567,8.0,431.625379,4.288574,57.900002,77.20
174,Mauritius,5182.143721,3.53,0.349544,31.6,5.947333e+07,36.299999,1.0,28.731883,73.373,NaN,10.0,NaN,8.081540,54.500000,42.48
175,Namibia,2667.246710,12.09,1.972913,24.7,2.620933e+07,37.299999,13.1,6.500823,62.465,NaN,6.0,0.000000,8.021970,42.000000,36.84
176,Comoros,336.368749,0.28,NaN,19.5,2.368667e+06,59.299999,0.1,5.098265,61.061,NaN,9.0,NaN,4.751084,68.000000,28.08


In [ ]:
duplicates_pais = pip_df.loc[pip_df["Pais"].duplicated(keep=False)]
duplicates_pais.shape

KeyError: ignored